In [1]:
from astropy.time import Time
import numpy as np 

""" Satellite orbital parameters """
VE = Time("2024-3-20 3:6:0") # current year's Vernal Equinox
T0 = VE
ecco = 0
argpo = 0 # degrees
inclo = 97.8 # degrees
RAAN = 270 # degrees
period = 100.9 # minutes
mo = 0 # degrees

""" Spacecraft body-frame vectors """
telescope_boresight = np.array([1, 0, 0])
solar_array = np.array([0, 0, -1])
antenna1 = np.array([0, np.sin(np.pi/6), np.cos(np.pi/6)])
antenna2 = np.array([0, np.sin(-np.pi/6), np.cos(-np.pi/6)])
antennas = [antenna1, antenna2]
antenna = antenna1

In [2]:
from satellite import Satellite
satellite = Satellite.build_from_orbit(T0, ecco, argpo, inclo, RAAN, period, mo)
satellite

/Users/michellekao/Desktop/castor_env/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


<EarthSatellite catalog #0 epoch 2024-03-20 03:06:00 UTC>

In [3]:
""" Downlink parameters """
# onboard data capacity
onboard_data_cap = 725 # in GB
# start searching for ground station when data onboard reaches this threshold
data_threshold = onboard_data_cap - 50 # in GB
# downlinking rate
downlink_rate = 10/8 # GBps

# a list of ground station locations 
ground_latlons = [(34.1478, -118.1445), (37.8213, 22.661), (35.6764, 139.65), 
                  (45.5037, -73.4292), (68.3194, -133.5492), (48.5204, -123.4188)]
ground_station_num = len(ground_latlons)
# maximum cloud coverage percentage allowed for optical downlinking
max_cloud_cover = 35 # %
# minimum angle above ground station horizon to perform downlink
elevation_min = 30 # degrees
# the antenna's maximum pivot angle from Nadir
pivot_angle = 62 # degrees

In [4]:
from targets import InertialTarget, SolarSystemTarget, EarthOrbitingTarget, EarthFixedTarget
from windows import Window
from astropy.time import Time
import astropy.units as u

import timeit
code_start = timeit.default_timer()

# Define ground station target objects
ground_targets = [EarthFixedTarget(lat, lon) for lat, lon in ground_latlons]

# Search for the downlink windows
downlink_windows = Window()
Now = Time('2024-10-30')
for station in ground_targets:
    downlink_windows += station.get_downlink_windows(Now, Now+1*u.day, max_cloud_cover, elevation_min, 
                                                     pivot_angle, antenna, telescope_boresight, satellite)

# Filter and merge the downlink windows
downlink_windows.filter(3*60).merge()

runtime = timeit.default_timer() - code_start
print('code runtime:', runtime/60, 'min')

downlink_windows.list

code runtime: 0.9153834361194944 min


[(<Time object: scale='utc' format='iso' value=2024-10-30 00:26:11.965>,
  <Time object: scale='utc' format='iso' value=2024-10-30 00:30:34.965>),
 (<Time object: scale='utc' format='iso' value=2024-10-30 07:09:57.611>,
  <Time object: scale='utc' format='iso' value=2024-10-30 07:13:36.611>),
 (<Time object: scale='utc' format='iso' value=2024-10-30 11:42:04.293>,
  <Time object: scale='utc' format='iso' value=2024-10-30 11:46:10.293>),
 (<Time object: scale='utc' format='iso' value=2024-10-30 18:26:25.737>,
  <Time object: scale='utc' format='iso' value=2024-10-30 18:30:17.737>)]

In [5]:
first = downlink_windows.list.pop(0)
first

(<Time object: scale='utc' format='iso' value=2024-10-30 00:26:11.965>,
 <Time object: scale='utc' format='iso' value=2024-10-30 00:30:34.965>)

In [6]:
downlink_windows.list

[(<Time object: scale='utc' format='iso' value=2024-10-30 07:09:57.611>,
  <Time object: scale='utc' format='iso' value=2024-10-30 07:13:36.611>),
 (<Time object: scale='utc' format='iso' value=2024-10-30 11:42:04.293>,
  <Time object: scale='utc' format='iso' value=2024-10-30 11:46:10.293>),
 (<Time object: scale='utc' format='iso' value=2024-10-30 18:26:25.737>,
  <Time object: scale='utc' format='iso' value=2024-10-30 18:30:17.737>)]

In [10]:
downlink_windows.update_num()
downlink_windows.num

3

In [14]:
a = Window([(1,1)]) 
a += Window([(2,2),(3,3)]) 
a.num

3